In [1]:
import os
import subprocess
import importlib
import pandas as pd
import tokenize
from matplotlib import pyplot as plt

In [2]:
import create_data_set_utils
import CodeImportLineCompleter
import CodePairImportLineCompleter
importlib.reload(create_data_set_utils)
importlib.reload(CodeImportLineCompleter)
importlib.reload(CodePairImportLineCompleter)

<module 'CodePairImportLineCompleter' from '/home/erez/research/code_completion/CodePairImportLineCompleter.py'>

# load from csv

In [85]:
imports_csv_file_path = 'imports.csv'
imports_tokens_df = pd.read_csv(imports_csv_file_path)
imports_tokens_df.head()

,end,file_path,line,line_index,start,string,type_name
0,"(1, 6)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1,"(1, 0)",import,NAME
1,"(1, 10)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1,"(1, 7)",sys,NAME
2,"(1, 11)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1,"(1, 10)",\n,NEWLINE
3,"(2, 6)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import os,2,"(2, 0)",import,NAME
4,"(2, 9)",/home/erez/github_repos/siva/latest/e2/temp_gi...,import os,2,"(2, 7)",os,NAME


# imports

In [86]:
imports_lines_df = imports_df.drop_duplicates(['file_path', 'line'])
imports_lines_df.shape

(56395, 7)

In [87]:
imports_lines_df.drop(columns=['end', 'start', 'type_name', 'string'], inplace=True)

/home/erez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [88]:
imports_lines_df.head()

,file_path,line,line_index
0,/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1
3,/home/erez/github_repos/siva/latest/e2/temp_gi...,import os,2
6,/home/erez/github_repos/siva/latest/e2/temp_gi...,from six.moves import urllib,3
13,/home/erez/github_repos/siva/latest/e2/temp_gi...,from scipy.io import loadmat,4
20,/home/erez/github_repos/siva/latest/e2/temp_gi...,import numpy as np,5


In [89]:
imports_counts_series = imports_lines_df.line.value_counts()
imports_counts_series.shape

(9026,)

In [90]:
imports_counts_df = pd.DataFrame(imports_counts_series)
imports_counts_df.rename(columns={'line': 'count'}, inplace=True)
imports_counts_df['line'] = imports_counts_df.index.str.strip()
imports_counts_df.reset_index(drop=True).head()

,count,line
0,3474,import numpy as np
1,3421,import tensorflow as tf
2,2803,from __future__ import print_function
3,2522,from __future__ import division
4,2455,from __future__ import absolute_import


# inline completer

In [81]:
inline_completer = CodeImportLineCompleter.CodeImportLineCompleter()
inline_completer.load_db(imports_lines_df.line)

In [82]:
inline_completer.complete_code('from __f')

97.91%


,count,line,prob
0,2803,from __future__ import print_function,0.352756
1,2522,from __future__ import division,0.317392
2,2455,from __future__ import absolute_import,0.308960


# based on previous 

In [59]:
imports_lines_df.head()

,file_path,line,line_index
0,/home/erez/github_repos/siva/latest/e2/temp_gi...,import sys,1
3,/home/erez/github_repos/siva/latest/e2/temp_gi...,import os,2
6,/home/erez/github_repos/siva/latest/e2/temp_gi...,from six.moves import urllib,3
13,/home/erez/github_repos/siva/latest/e2/temp_gi...,from scipy.io import loadmat,4
20,/home/erez/github_repos/siva/latest/e2/temp_gi...,import numpy as np,5


In [60]:
len(imports_lines_df.file_path.unique())

8985

In [61]:
len(imports_lines_df.line.unique())

9026

In [77]:
with_previous_line_completer = CodePairImportLineCompleter.CodePairImportLineCompleter()
with_previous_line_completer.load_db(imports_lines_df)

In [78]:
with_previous_line_completer.complete_code('import numpy as np', 5)

23.84%


,count,line,prob
0,1602,import tensorflow as tf,0.068185
1,1096,from __future__ import print_function,0.046648
2,1012,import os,0.043073
3,970,from __future__ import division,0.041285
4,922,from __future__ import absolute_import,0.039242


In [79]:
with_previous_line_completer.complete_code('import shutil', 5)

23.61%


,count,line,prob
0,147,import os,0.078863
1,82,from __future__ import print_function,0.043991
2,73,from __future__ import absolute_import,0.039163
3,72,import tensorflow as tf,0.038627
4,66,from __future__ import division,0.035408


# joint inline and previous line

In [69]:
string = \
"""import numpy as np
import p"""
print(string)

import numpy as np
import p


In [76]:
previous_line, current_line = string.split('\n')
previous_line, current_line

('import numpy as np', 'import p')

In [84]:
with_previous_line_completer.complete_code(previous_line, 100)

58.37%


,count,line,prob
0,1602,import tensorflow as tf,0.068185
1,1096,from __future__ import print_function,0.046648
2,1012,import os,0.043073
3,970,from __future__ import division,0.041285
4,922,from __future__ import absolute_import,0.039242
5,457,import sys,0.019451
6,448,import time,0.019068
7,421,import torch,0.017919
8,342,import random,0.014556
9,310,import math,0.013194


In [83]:
inline_completer.complete_code(current_line)

45.68%


,count,line,prob
0,389,import pandas as pd,0.247141
1,200,import pandas.util.testing as tm,0.127065
2,130,import pickle,0.082592
